<a href="https://colab.research.google.com/github/ShunMatsuoka/regression_search/blob/main/gbm_fix.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [87]:
from sklearn.datasets import load_boston
import numpy as np

dataset = load_boston()

In [88]:
x = dataset.data #説明変数
t = dataset.target #目的変数

In [89]:
f_name = dataset.feature_names

In [90]:
f_name

array(['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD',
       'TAX', 'PTRATIO', 'B', 'LSTAT'], dtype='<U7')

In [91]:
x.shape

(506, 13)

In [92]:
#データセットを分割する関数の読み込み
from sklearn.model_selection import train_test_split
#訓練用データとテストデータセットへの分割
x_train,x_test,t_train,t_test = train_test_split(x,t,test_size=0.3,random_state=0)

In [93]:
import pandas as pd
#データフレーム型に変換
train_df = pd.DataFrame(x_train,columns=f_name)
#不要カラムの削除
train_df = train_df.drop(["ZN", "CHAS", "RAD"], axis=1)
train_df

,CRIM,INDUS,NOX,RM,AGE,DIS,TAX,PTRATIO,B,LSTAT
0,1.62864,21.89,0.624,5.019,100.0,1.4394,437.0,21.2,396.90,34.41
1,0.11460,6.96,0.464,6.538,58.7,3.9175,223.0,18.6,394.96,7.73
2,0.55778,21.89,0.624,6.335,98.2,2.1107,437.0,21.2,394.67,16.96
3,0.06466,2.24,0.400,6.345,20.1,7.8278,358.0,14.8,368.24,4.97
4,0.09299,25.65,0.581,5.961,92.9,2.0869,188.0,19.1,378.09,17.93
...,...,...,...,...,...,...,...,...,...,...
349,0.28392,7.38,0.493,5.708,74.3,4.7211,287.0,19.6,391.13,11.74
350,0.08664,3.44,0.437,7.178,26.3,6.4798,398.0,15.2,390.49,2.87
351,0.15098,10.01,0.547,6.021,82.6,2.7474,432.0,17.8,394.51,10.30
352,0.22927,6.91,0.448,6.030,85.5,5.6894,233.0,17.9,392.74,18.80


In [94]:
train_df = pd.concat([train_df,pd.DataFrame(t_train)],axis=1).rename(columns={0:"MEDV"})

In [95]:
train_df

,CRIM,INDUS,NOX,RM,AGE,DIS,TAX,PTRATIO,B,LSTAT,MEDV
0,1.62864,21.89,0.624,5.019,100.0,1.4394,437.0,21.2,396.90,34.41,14.4
1,0.11460,6.96,0.464,6.538,58.7,3.9175,223.0,18.6,394.96,7.73,24.4
2,0.55778,21.89,0.624,6.335,98.2,2.1107,437.0,21.2,394.67,16.96,18.1
3,0.06466,2.24,0.400,6.345,20.1,7.8278,358.0,14.8,368.24,4.97,22.5
4,0.09299,25.65,0.581,5.961,92.9,2.0869,188.0,19.1,378.09,17.93,20.5
...,...,...,...,...,...,...,...,...,...,...,...
349,0.28392,7.38,0.493,5.708,74.3,4.7211,287.0,19.6,391.13,11.74,18.5
350,0.08664,3.44,0.437,7.178,26.3,6.4798,398.0,15.2,390.49,2.87,36.4
351,0.15098,10.01,0.547,6.021,82.6,2.7474,432.0,17.8,394.51,10.30,19.2
352,0.22927,6.91,0.448,6.030,85.5,5.6894,233.0,17.9,392.74,18.80,16.6


In [96]:
train_df.describe().T

,count,mean,std,min,25%,50%,75%,max
CRIM,354.0,3.358284,8.353223,0.00632,0.073425,0.26266,3.10370,88.9762
INDUS,354.0,11.078757,6.993821,0.46000,4.950000,8.56000,18.10000,27.7400
NOX,354.0,0.556098,0.115601,0.38500,0.450000,0.53800,0.62925,0.8710
RM,354.0,6.308427,0.702009,3.56100,5.895250,6.21550,6.64725,8.7800
AGE,354.0,68.994068,28.038429,2.90000,45.175000,79.45000,93.75000,100.0000
DIS,354.0,3.762459,2.067661,1.17420,2.107650,3.21570,5.07930,12.1265
TAX,354.0,401.782486,170.592404,187.00000,276.000000,311.00000,666.00000,711.0000
PTRATIO,354.0,18.473446,2.224809,12.60000,17.400000,19.10000,20.20000,22.0000
B,354.0,360.601186,85.621945,0.32000,376.057500,391.60500,395.69000,396.9000
LSTAT,354.0,12.440650,7.078485,1.73000,6.735000,11.16000,16.71750,36.9800


# 前処理に標準化を入れる

In [111]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [112]:
scaler.fit(x_train)

StandardScaler(copy=True, with_mean=True, with_std=True)

In [115]:
x_train_scaled = scaler.transform(x_train)
x_test_scaled = scaler.transform(x_test)

# GradientBoostingRegressor


In [116]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV

In [117]:
GBR = GradientBoostingRegressor()
parameters = {'learning_rate' : [0.1, 0.05, 0.02, 0.01], 
              'max_depth': [4, 6],
              'min_samples_leaf': [3, 5, 9, 17],
              'max_features': [1.0, 0.3, 0.1]}

In [118]:
GBR = GridSearchCV(GBR, parameters, verbose=10, n_jobs=-1, cv=4)
GBR.fit(x_train_scaled, t_train)
print("精度の検証訓練データ{}".format(GBR.score(x_train_scaled,t_train)))
print("精度の検証テストデータ{}".format(GBR.score(x_test_scaled,t_test)))

Fitting 4 folds for each of 96 candidates, totalling 384 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    1.2s
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:    1.5s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    2.0s
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed:    2.4s
[Parallel(n_jobs=-1)]: Done  21 tasks      | elapsed:    2.8s
[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed:    3.1s
[Parallel(n_jobs=-1)]: Batch computation too fast (0.1998s.) Setting batch_size=2.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    3.7s
[Parallel(n_jobs=-1)]: Done  60 tasks      | elapsed:    5.3s
[Parallel(n_jobs=-1)]: Done  82 tasks      | elapsed:    6.4s
[Parallel(n_jobs=-1)]: Done 104 tasks      | elapsed:    7.8s
[Parallel(n_jobs=-1)]: Done 130 tasks      | elapsed:    9.1s
[Parallel(n_jobs=-1)]: Done 156 tasks      | elapsed:   11.0s
[Parallel(n_jobs=-1)]: Done 186 tasks      | elapsed:   12.5s
[Parallel(n_jobs=-1)]: Done 216 tas

精度の検証訓練データ0.9888400666057614
精度の検証テストデータ0.7696634463482991


[Parallel(n_jobs=-1)]: Done 384 out of 384 | elapsed:   24.2s finished


In [119]:
GBR.fit(x_train, t_train)
print("精度の検証訓練データ{}".format(GBR.score(x_train,t_train)))
print("精度の検証テストデータ{}".format(GBR.score(x_test,t_test)))

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


Fitting 4 folds for each of 96 candidates, totalling 384 fits


[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Batch computation too fast (0.1817s.) Setting batch_size=2.
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:    0.4s
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed:    1.4s
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:    1.8s
[Parallel(n_jobs=-1)]: Done  38 tasks      | elapsed:    2.5s
[Parallel(n_jobs=-1)]: Done  52 tasks      | elapsed:    3.3s
[Parallel(n_jobs=-1)]: Done  70 tasks      | elapsed:    4.9s
[Parallel(n_jobs=-1)]: Done  88 tasks      | elapsed:    5.7s
[Parallel(n_jobs=-1)]: Done 110 tasks      | elapsed:    7.2s
[Parallel(n_jobs=-1)]: Done 132 tasks      | elapsed:    8.3s
[Parallel(n_jobs=-1)]: Done 158 tasks      | elapsed:   10.3s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:   11.5s
[Parallel(n_jobs=-1)]: Done 214 tasks      | elapsed:   13.5s
[Parallel(n_jobs=-1)]: Done 244 tasks      | elapsed:   14.9s
[Parallel(n_jobs=-1)]: Done 278 tasks      | elap

精度の検証訓練データ0.9973360252256227
精度の検証テストデータ0.7573341266769221


[Parallel(n_jobs=-1)]: Done 384 out of 384 | elapsed:   23.2s finished


In [120]:
model = GBR.best_estimator_
model.set_params(n_estimators=100000)
parameters = {'learning_rate' : [0.1, 0.05, 0.02, 0.01]}
gscv = GridSearchCV(model, parameters, verbose=10, n_jobs=-1, cv=4)
gscv.fit(x_train_scaled, t_train)
print("best score=", gscv.best_score_)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


Fitting 4 folds for each of 4 candidates, totalling 16 fits


[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:   53.0s
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:  4.4min
[Parallel(n_jobs=-1)]: Done  16 out of  16 | elapsed:  7.2min finished


best score= 0.8968217341313686


In [121]:
gscv.best_score_

0.8968217341313686

In [ ]:
a = model.predict(x_test)

In [122]:
model.predict(x_test)

array([24.09722345, 24.97172397, 22.68155099,  9.72882427, 21.15958564,
       20.7127432 , 20.97682609, 20.07141527, 21.56598203, 18.3239289 ,
        8.7088984 , 17.27440694, 14.44527906,  8.12411214, 49.35122607,
       33.63143539, 20.98180342, 36.77638094, 26.16766969, 22.03054558,
       23.24180093, 22.91005755, 19.66582792, 24.92991781, 20.79107419,
       19.85182271, 17.34871298, 16.95802337, 41.45266908, 17.46300911,
       17.20530534, 16.76642576, 19.65159523, 20.23855219, 23.88523988,
       17.17849302,  8.42663433, 19.64849908, 13.56305295, 15.70227131,
       23.33644455, 20.55126154, 21.90907065, 15.36829496, 21.86074148,
       21.05491107, 21.967707  , 16.93213103, 19.13341444, 24.22733876,
       15.82279472, 20.9521706 , 20.62886628, 34.4592054 , 14.57890945,
       19.99843324, 19.25125338, 17.79811934, 18.71086012, 19.98025367,
       22.13497907, 20.7285306 , 29.43626523, 34.2904132 , 19.80050198,
       30.48119872, 17.31548283, 18.78642024, 16.04003448, 23.41